In [96]:
from tqdm import tqdm

In [97]:
import pandas as pd
import numpy as np
path = './data/road_info.csv'
road_info = pd.read_csv(path)
labled_tracks = pd.read_csv('./data/labled_train_tracks_speed.csv')
unlabled_tracks = pd.read_csv('./data/unlabled_train_tracks_speed.csv')

In [98]:
road_info['lat'] = road_info['loc'].apply(lambda x: round(eval(x)[0], 3))
road_info['lon'] = road_info['loc'].apply(lambda x: round(eval(x)[1], 3))
labled_tracks['lat']=labled_tracks['lat_'].apply(lambda x: round(x, 3))
labled_tracks['lon']=labled_tracks['lon_'].apply(lambda x: round(x, 3))

labled_tracks

,Unnamed: 0,Unnamed: 0.1,driver_id,dt,lat_,lon_,order_id,speed,is_aggressive,lat,lon
0,0,0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:43:41,55.792710,37.545409,001662da857b5a39bb402aacf3145f86,NaN,0.0,55.793,37.545
1,1,1,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:44:40,55.792013,37.544481,001662da857b5a39bb402aacf3145f86,5.906441,0.0,55.792,37.544
2,2,2,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:00,55.791365,37.543695,001662da857b5a39bb402aacf3145f86,15.696000,0.0,55.791,37.544
3,3,3,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:20,55.791267,37.543512,001662da857b5a39bb402aacf3145f86,2.862000,0.0,55.791,37.544
4,4,4,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:37,55.791175,37.543634,001662da857b5a39bb402aacf3145f86,2.710588,0.0,55.791,37.544
...,...,...,...,...,...,...,...,...,...,...,...
395682,675217,675217,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:38:08,55.833583,37.490335,fffdfda358f35cf2d7f9d87d205655a7,36.891429,0.0,55.834,37.490
395683,675219,675219,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:38:57,55.833093,37.491756,fffdfda358f35cf2d7f9d87d205655a7,8.517273,0.0,55.833,37.492
395684,675220,675220,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:39:14,55.832650,37.493704,fffdfda358f35cf2d7f9d87d205655a7,27.783529,0.0,55.833,37.494
395685,675222,675222,1c11e4febcc32f17e7fbbd20a86b9a59,2021-03-04 19:39:37,55.833031,37.495477,fffdfda358f35cf2d7f9d87d205655a7,23.720000,0.0,55.833,37.495


In [99]:
df = road_info.merge(labled_tracks, on=['lat', 'lon'], how='right')
df.count()

id                 381644
maxspeed           104795
lanes              233570
road_type          381644
loc                381644
traffic_signals    381644
lat                614652
lon                614652
Unnamed: 0         614652
Unnamed: 0.1       614652
driver_id          614652
dt                 614652
lat_               614652
lon_               614652
order_id           614652
speed              614651
is_aggressive      614652
dtype: int64

In [100]:
df.order_id.describe()

count                               614652
unique                                9000
top       c6da33b2ac344d7d7165454eded04104
freq                                   555
Name: order_id, dtype: object

In [101]:
dic = {'ru:urban': 60, 'ru:living_street': 20, 'ru:rural': 90, 'ru:motorway':120}
mxspeed = df.maxspeed.apply(lambda x: int(x) if type(x)==str and x.isdigit() else x)
df.maxspeed =  mxspeed.apply(lambda x: dic[x.lower()] if type(x)==str else x)



In [103]:
features = df
features['is_violation'] = np.where(features.speed>features.maxspeed, True, False)
features['near_violation'] = np.where((features.speed-features.maxspeed)<0.5, True, False)

groupedby_order= features.groupby('order_id')

count_violations = groupedby_order.apply(lambda x: x.is_violation.sum())
count_near_violations = groupedby_order.apply(lambda x: x.near_violation.sum())

tosave = pd.DataFrame({'violations':count_violations, 'near_violations': count_near_violations})
tosave.to_csv('abc.csv')
#features.near_violation.sum()
#np.where((features[features['near_violation']==True] == features[features['is_aggressive']==True]), True, False).sum()
#features['aggressive_and_near']=features.loc[features['is_aggressive']==1,] and features.loc[features['near_violation']==True,]
all_aggressive = features.is_aggressive.count()
aggressive = features.is_aggressive.sum()

# пересекающиеся
from collections import Counter
#counter(features[])
peresek = np.where((features['is_violation']==True) & (features['is_aggressive']==1), True, False).sum()
a = np.where((features['is_violation']==True) & (features['is_aggressive']==1), True, False)
print(Counter(a)[False])
peresek2 = np.where((features['near_violation']==True) & (features['is_aggressive']==0), True, False)
print(Counter(peresek2))

print(peresek/aggressive)
peresek2/(all_aggressive-aggressive)

614123
Counter({False: 524802, True: 89850})
0.014683024314422116


array([0., 0., 0., ..., 0., 0., 0.])